<a href="https://colab.research.google.com/github/CamiloVga/Codes/blob/main/BotAmbientalV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers huggingface_hub gradio
!pip install -U bitsandbytes
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/

In [2]:


# Import necessary packages
from transformers import pipeline, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import hf_hub_download
import gradio as gr

# Define the model name
model_name = "anslin-raj/Llama-2-7b-chat-hf-8-bit"

# Set up quantization configuration
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True  # Add this if the model uses custom code
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Create the text-generation pipeline
model_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
)




config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


pytorch_model.bin:   0%|          | 0.00/7.01G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [3]:
# Configurar el mensaje del sistema
mensaje_sistema = {
    "role": "system",
    "content": "Eres un chatbot experto en economía. Proporciona información perspicaz y precisa sobre economía, finanzas y tendencias del mercado. Utiliza terminología económica apropiadamente y explica conceptos complejos de forma clara."
}

# Inicializar variables
mensajes = [mensaje_sistema]
consumo_agua_por_pregunta = 25  # 500 ml / 20 preguntas
consumo_total_agua = 0  # en ml

def generar_respuesta(entrada_usuario, historial_chat):
    global consumo_total_agua, mensajes

    # Agregar la entrada del usuario a los mensajes
    mensajes.append({"role": "user", "content": entrada_usuario})

    # Crear el prompt
    prompt = ""
    for m in mensajes:
        if m["role"] == "system":
            prompt += f"<INICIO MENSAJE SISTEMA>\n{m['content']}\n<FIN MENSAJE SISTEMA>\n\n"
        elif m["role"] == "user":
            prompt += f"Usuario: {m['content']}\n"
        else:
            prompt += f"Asistente: {m['content']}\n"
    prompt += "Asistente:"

    # Generar la respuesta
    salidas = model_gen(
        prompt,
        max_new_tokens=256,
        return_full_text=False
    )

    # Extraer la respuesta del asistente
    respuesta_asistente = salidas[0]['generated_text'].strip()

    # Agregar la respuesta del asistente a los mensajes
    mensajes.append({"role": "assistant", "content": respuesta_asistente})

    # Actualizar consumo total de agua
    consumo_total_agua += consumo_agua_por_pregunta

    # Actualizar historial del chat
    historial_chat.append((entrada_usuario, respuesta_asistente))

    # Preparar mensaje de consumo de agua
    mensaje_agua = f"Consumo total de agua: {consumo_total_agua} ml"

    return historial_chat, mensaje_agua

# Crear la interfaz de Gradio
with gr.Blocks() as demo:
    gr.Markdown("<h1>Chatbot de Conocimientos Económicos</h1>")
    gr.Markdown("Bienvenido al Chatbot de Conocimientos Económicos. ¿En qué puedo ayudarte hoy con temas económicos?")

    # Crear componentes del chat
    chatbot = gr.Chatbot()
    mensaje = gr.Textbox(placeholder="Escribe tu pregunta aquí...")
    mostrar_agua = gr.Markdown(f"Consumo total de agua: {consumo_total_agua} ml")
    limpiar = gr.Button("Limpiar Chat")

    # Definir la acción de envío
    def enviar(entrada_usuario, historial_chat):
        return generar_respuesta(entrada_usuario, historial_chat)

    # Configurar los manejadores de eventos
    mensaje.submit(enviar, [mensaje, chatbot], [chatbot, mostrar_agua])
    limpiar.click(lambda: ([], f"Consumo total de agua: 0 ml"), None, [chatbot, mostrar_agua])

# Lanzar la aplicación
demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://805b36c696aa4fca76.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
